In [4]:
import numpy as np
import scipy.sparse
import sys
import tensorflow as tf

In [5]:
sys.path.append("article_finder")
import articles
import embeddings

## Reading embeddings

In [7]:
def read_words_embeddings():
    with open("article_finder/data/wiki_embeddings_300.txt") as file_stream:
        lines = [x.strip() for x in file_stream.readlines()]
        return {x.split()[0].lower(): np.fromstring(" ".join(x.split()[1:]), dtype=float, sep=" ") for x in lines[1:]}
    

words_embeddings = read_words_embeddings()

## Articles terms array

In [9]:
words_base_forms = embeddings.read_words_base_forms(path="article_finder/data/base_forms.txt")
wiki_articles = articles.read_wiki_articles(path="article_finder/data/wiki_slice.txt")
rows_article_ids = dict(enumerate([x.id for x in sorted(wiki_articles, key=lambda x: x.id)]))
article_ids_rows = dict([(y, x) for x, y in rows_article_ids.items()])
titles_wiki_articles = {wiki_article.title: wiki_article for wiki_article in wiki_articles}
ids_wiki_articles = {wiki_article.id: wiki_article for wiki_article in wiki_articles}
terms_indexes = embeddings.get_terms_indexes(words_base_forms, wiki_articles, add_missing_forms=True)
indexes_terms = {x_y[1]: x_y[0] for x_y in terms_indexes.items()}
idf_vector = embeddings.create_idf_vector(words_base_forms, wiki_articles, terms_indexes)
terms_idf_values = {indexes_terms[i]: idf_vector[i] for i in range(idf_vector.shape[0])}

In [10]:
banned_terms = {indexes_terms[index] for index in np.argsort(idf_vector.reshape(-1, ))[:100]}

In [11]:
TOP_TERMS_COUNT = 20
articles_terms_array = (-1) * np.ones((len(wiki_articles), TOP_TERMS_COUNT), dtype=np.int32)
prefix_terms_indexes = {}
for wiki_article in wiki_articles:
    article_terms = set()
    for word in wiki_article.content.split(" "):
        for term in words_base_forms[word]:
            if len(article_terms) >= TOP_TERMS_COUNT:
                break
            if term not in terms_indexes or term in banned_terms or term in article_terms or term not in words_embeddings:
                continue
            if term not in prefix_terms_indexes:
                prefix_terms_indexes[term] = len(prefix_terms_indexes)
            articles_terms_array[article_ids_rows[wiki_article.id], len(article_terms)] = prefix_terms_indexes[term]
            article_terms.add(term)
            
            
prefix_indexes_terms = {y: x for (x, y) in prefix_terms_indexes.items()}

In [12]:
article_terms = set(articles_terms_array[article_ids_rows[3097]])

In [13]:
for index, row in enumerate(articles_terms_array):
    if len(set(row).intersection(article_terms)) >= 7:
        print(ids_wiki_articles[rows_article_ids[index]].title)

C++
Python
Język wysokiego poziomu
Terminator (programowanie)
CodeMirror
Ymacs
Lisp-maszyna


In [14]:
titles_wiki_articles["Python"]

WikiArticle(id=3097, title='Python', content='Python\nPython – język programowania wysokiego poziomu ogólnego przeznaczenia , o rozbudowanym pakiecie bibliotek standardowych , którego ideą przewodnią jest czytelność i klarowność kodu źródłowego . Jego składnia cechuje się przejrzystością i zwięzłością .\nPython wspiera różne paradygmaty programowania : obiektowy , imperatywny oraz w mniejszym stopniu funkcyjny . Posiada w pełni dynamiczny system typów i automatyczne zarządzanie pamięcią , będąc w tym podobnym do języków Perl , Ruby , Scheme czy Tcl . Podobnie jak inne języki dynamiczne jest często używany jako język skryptowy . Interpretery Pythona są dostępne na wiele systemów operacyjnych .')

## Creating articles embeddings

In [22]:
rows_embeddings = []
for index, row in enumerate(articles_terms_array):
    if any([row[i] == -1 for i in range(len(row))]):
        rows_embeddings.append(np.zeros(300))   
        continue
    article_terms = [prefix_indexes_terms[row[i]] for i in range(len(row))]
    article_embeddings = []
    for index, term in enumerate(article_terms):
        term_embedding = words_embeddings[term] / np.linalg.norm(words_embeddings[term])
        weight = np.exp(-0.1 * index) 
        article_embeddings.append(weight * term_embedding)
    rows_embeddings.append(np.mean(article_embeddings, axis=0))    

In [23]:
scipy.sparse.save_npz(
    "article_finder/data/dense_embeddings_300.npz",
    scipy.sparse.csr_matrix(np.array(rows_embeddings))
)